### Import Dependencies

In [1]:
import numpy as np 
import pandas as pd
import mediapipe as mp

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from datetime import datetime
from packaging import version

# Load the TensorBoard notebook extension.
%load_ext tensorboard

### Read Dataset 

In [2]:
data = pd.read_csv('../data.csv')
data.drop('Index', axis=1, inplace=True)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,459,460,461,462,463,464,465,466,467,label
0,0.156041,0.034724,0.071444,0.078117,0.0,0.043656,0.147673,0.290654,0.227140,0.271585,...,0.063550,0.104668,0.057380,0.062866,0.189442,0.170856,0.159821,0.323322,0.348375,0
1,0.099105,0.033467,0.048210,0.096153,0.0,0.050459,0.181863,0.332776,0.276899,0.323902,...,0.061139,0.101462,0.051161,0.050882,0.230257,0.214284,0.201468,0.353529,0.377521,0
2,0.160259,0.033538,0.105772,0.086060,0.0,0.046421,0.181095,0.224147,0.273403,0.312137,...,0.071666,0.147298,0.067166,0.087085,0.274896,0.251641,0.229446,0.382837,0.408696,0
3,0.139705,0.035719,0.060722,0.098022,0.0,0.051891,0.187796,0.330197,0.296981,0.350095,...,0.063510,0.106100,0.055346,0.057549,0.232088,0.215398,0.203230,0.354787,0.383226,0
4,0.144930,0.030739,0.065995,0.082237,0.0,0.041594,0.145923,0.312308,0.210267,0.247076,...,0.035856,0.051338,0.042271,0.047047,0.154600,0.146428,0.139789,0.194182,0.204608,0


### Split variables from label

In [3]:
x = data.iloc[:, :-1]
y = data.iloc[:, -1]


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1) 

### Initial Model

In [5]:
logdir = "./logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [6]:
inputs = keras.Input(shape = 468)

z = layers.Dense(24, activation="relu")(inputs)
outputs = layers.Dense(6, activation="softmax")(z)

model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 468)]             0         
                                                                 
 dense (Dense)               (None, 24)                11256     
                                                                 
 dense_1 (Dense)             (None, 6)                 150       
                                                                 
Total params: 11,406
Trainable params: 11,406
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy'],
)

batch_size = 32
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=batch_size,
    verbose=1, 
    epochs=50,
    shuffle=True,
    validation_data=(x_val, y_val),
    callbacks=[callback, tensorboard_callback],
)


Epoch 1/50
610/610 [==============================] - 2s 2ms/step - loss: 1.7446 - accuracy: 0.2643 - val_loss: 1.7314 - val_accuracy: 0.2625
Epoch 2/50
610/610 [==============================] - 1s 2ms/step - loss: 1.7170 - accuracy: 0.2693 - val_loss: 1.6994 - val_accuracy: 0.2912
Epoch 3/50
610/610 [==============================] - 1s 2ms/step - loss: 1.6796 - accuracy: 0.2923 - val_loss: 1.6647 - val_accuracy: 0.3572
Epoch 4/50
610/610 [==============================] - 1s 2ms/step - loss: 1.6422 - accuracy: 0.3266 - val_loss: 1.6159 - val_accuracy: 0.3443
Epoch 5/50
610/610 [==============================] - 1s 2ms/step - loss: 1.6058 - accuracy: 0.3516 - val_loss: 1.5804 - val_accuracy: 0.3375
Epoch 6/50
610/610 [==============================] - 1s 2ms/step - loss: 1.5745 - accuracy: 0.3704 - val_loss: 1.5636 - val_accuracy: 0.3412
Epoch 7/50
610/610 [==============================] - 1s 2ms/step - loss: 1.5510 - accuracy: 0.3827 - val_loss: 1.5238 - val_accuracy: 0.4063
Epoch 

In [8]:
predictions = model.predict(x_test)
predictions = keras.backend.argmax(
    predictions,
    axis=-1
)


In [9]:
accuracy_score(y_test, predictions)

0.44753846153846155

# pca

In [10]:
from sklearn.decomposition import PCA

pca = PCA(.99)
pca.fit(x_train)

PCA(n_components=0.99)

In [11]:
train = pca.transform(x_train)
test = pca.transform(x_test)
val = pca.transform(x_val)


In [12]:
train.shape

(19498, 7)

In [13]:
logdir = "./logs/pca" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

inputs1 = keras.Input(shape = 468)

r = layers.Dense(24, activation="relu")(inputs1)
outputs1 = layers.Dense(6, activation="softmax")(r)

model1 = keras.Model(inputs1, outputs1)
model1.summary()

model1.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy'],
)

batch_size = 64
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)

history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=batch_size,
    verbose=1, 
    epochs=500,
    shuffle=True,
    validation_data=(x_val, y_val),
    callbacks=[tensorboard_callback],
)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 468)]             0         
                                                                 
 dense_2 (Dense)             (None, 24)                11256     
                                                                 
 dense_3 (Dense)             (None, 6)                 150       
                                                                 
Total params: 11,406
Trainable params: 11,406
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
305/305 [==============================] - 1s 2ms/step - loss: 1.3940 - accuracy: 0.4503 - val_loss: 1.4041 - val_accuracy: 0.4509
Epoch 2/500
305/305 [==============================] - 1s 2ms/step - loss: 1.3922 - accuracy: 0.4521 - val_loss: 1.3929 - val_accuracy: 0.4554
Epoch 3/500
305/305 [============